**10. This question should be answered using the Carseats data set.**

**(a) Fit a multiple regression model to predict **Sales** using:<br>**
- **Price**<br>
- **Urban**<br>
- **US**

In [65]:
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
from statsmodels.graphics.regressionplots import influence_plot

In [27]:
from ISLP import load_data
carseats = load_data('Carseats')
carseats.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [ ]:
carseats = pd.get_dummies(carseats,columns=["Urban","US"],drop_first=True)
carseats["Urban_Yes"] = carseats["Urban_Yes"].astype(int)
carseats["US_Yes"] = carseats["US_Yes"].astype(int)
carseats.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban_Yes,US_Yes
0,9.50,138,73,11,276,120,Bad,42,17,1,1
1,11.22,111,48,16,260,83,Good,65,10,1,1
2,10.06,113,35,10,269,80,Medium,59,12,1,1
3,7.40,117,100,4,466,97,Medium,55,14,1,1
4,4.15,141,64,3,340,128,Bad,38,13,1,0


In [42]:
x = carseats.loc[:,["Price","Urban_Yes","US_Yes"]]
x =sm.add_constant(x)
y = carseats.Sales

reg = sm.OLS(y,x).fit()
reg.summary()  


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Sales   R-squared:                       0.239
Model:                            OLS   Adj. R-squared:                  0.234
Method:                 Least Squares   F-statistic:                     41.52
Date:                Tue, 29 Apr 2025   Prob (F-statistic):           2.39e-23
Time:                        21:01:09   Log-Likelihood:                -927.66
No. Observations:                 400   AIC:                             1863.
Df Residuals:                     396   BIC:                             1879.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.0435      0.651     20.036      0.000      11.764      14.323
Price         -0.0545      0.005    -10.389      0.000      -0.065      -0.044
Urban_Yes     -0.0219      0.272     -0.081      0.936      -0.556       0.512
US_Yes         1.2006      0.259      4.635      0.000       0.691       1.710
==============================================================================
Omnibus:                        0.676   Durbin-Watson:                   1.912
Prob(Omnibus):                  0.713   Jarque-Bera (JB):                0.758
Skew:                           0.093   Prob(JB):                        0.684
Kurtosis:                       2.897   Cond. No.                         628.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**(b) Provide an interpretation of each coefficient in the model. Be careful—some of the variables in the model are qualitative!**  

- **const (13.04):** Baseline sales = **13,043 units** (Price=$0, Urban="No", US="No").  
- **Price (-0.055):** For every **$1 Increases**, sales **drop by 55 units**  
- **Urban_Yes (-0.02):** Urban stores sell **22 fewer units** than non-urban stores (base)  
- **US_Yes (1.20):** US stores sell **1,201 more units** than non-US stores (base)   
Dummy Base Levels:** "No" for both Urban and US.  

**(c) Write out the model in equation form, being careful to handle
the qualitative variables properly.**

$$ \text{Sales} = 13.0435 - 0.0545 \times \text{Price} - 0.0219 \times \text{Urban\_Yes} + 1.2006 \times \text{US\_Yes} $$


**(d) For which of the predictors can you reject the null hypothesis?**

$$ H_0: \beta_j = 0 $$



- **Price** (p < 0.001) → Significant  
- **US_Yes** (p < 0.001) → Significant  
- **Urban_Yes** (p = 0.936) → Not significant  

**Conclusion:** We reject H₀ for **Price** and **US location**, but not for Urban status.

**(e) On the basis of your response to the previous question, fit a
smaller model that only uses the predictors for which there is
evidence of association with the outcome.**

In [46]:
new_x = x.drop(columns=["Urban_Yes"])
new_x = sm.add_constant(new_x)

new_reg = sm.OLS(y,new_x).fit()
new_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Sales   R-squared:                       0.239
Model:                            OLS   Adj. R-squared:                  0.235
Method:                 Least Squares   F-statistic:                     62.43
Date:                Tue, 29 Apr 2025   Prob (F-statistic):           2.66e-24
Time:                        21:13:52   Log-Likelihood:                -927.66
No. Observations:                 400   AIC:                             1861.
Df Residuals:                     397   BIC:                             1873.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.0308      0.631     20.652      0.000      11.790      14.271
Price         -0.0545      0.005    -10.416      0.000      -0.065      -0.044
US_Yes         1.1996      0.258      4.641      0.000       0.692       1.708
==============================================================================
Omnibus:                        0.666   Durbin-Watson:                   1.912
Prob(Omnibus):                  0.717   Jarque-Bera (JB):                0.749
Skew:                           0.092   Prob(JB):                        0.688
Kurtosis:                       2.895   Cond. No.                         607.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**(f) How well do the models in (a) and (e) fit the data?**

In [54]:
print(f"R-squared: {reg.rsquared:.4f}")
print(f"new R-squared: {new_reg.rsquared:.4f}")

R-squared: 0.2393
new R-squared: 0.2393


Removing the Urban from the first model, there is no change in the R-squared value in the second model. Through this we can also   
conclude that Urban has no say in the prediction of the response, hence its better to use the model with two predictors

**(g) Using the model from (e), obtain 95 % confidence intervals for
the coefficient(s).**

In [61]:
Confidence_interval_95 = new_reg.conf_int(0.05).rename(columns={0:"lower_bound",1:"upper_bound"})
Confidence_interval_95 = Confidence_interval_95.drop(index=["const"])
print(Confidence_interval_95)

        lower_bound  upper_bound
Price      -0.06476    -0.044195
US_Yes      0.69152     1.707766


**(h) Is there evidence of outliers or high leverage observations in the
model from (e)?**

#pore korbo